In [38]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, Trainer, TrainingArguments
import evaluate
import numpy as np

In [39]:
import os
print(os.getcwd())

/mnt/c/Users/leonl/OneDrive/College/Senior/CSE 481DS/Analysis


In [40]:
def to_bool(string:str) -> bool:
    string = str(string).strip().casefold()
    if string == 'nan':
        return False
    if string in ('true', 'yes', '1'):
        return True
    if string in ('false', 'no', '0'):
        return False
    return False

def dsmap(ds, col, fn):
    # I hate huggingface datasets
    return ds.map(lambda x: {**x, **{col: fn(x[col])}})

In [46]:
# Load trainset from ../irr/train_set_gpt/caitlyn.csv
trainset = load_dataset("csv", data_files="./classify/irr/test_set_gpt/caitlyn.csv")["train"]
testset  = load_dataset("csv", data_files="./classify/irr/train_set_human/leon.csv")["train"]

# Set all columns to bool
trainset = dsmap(trainset, 'BreachMentioned', lambda x: int(to_bool(x)))
trainset = dsmap(trainset, 'CompanyMentioned', lambda x: int(bool(x)))
testset  = dsmap(testset, 'BreachMentioned', lambda x: int(to_bool(x)))
testset  = dsmap(testset, 'CompanyMentioned', lambda x: int(bool(x)))

# Remove non-input, non-target columns
trainset = trainset.remove_columns(['Date', 'Publication', 'URL'] + [x for x in trainset.column_names if x.startswith('Unnamed: ')])
testset  = testset.remove_columns(['Date', 'Publication', 'URL'] + [x for x in testset.column_names if x.startswith('Unnamed: ')])

# Merge BreachMentioned and CompanyMentioned into a single column
trainset = trainset.map(lambda e: {'Target': [int(bool(e['BreachMentioned'])), int(bool(e['CompanyMentioned']))]})
testset  = testset.map(lambda e: {'Target': [int(bool(e['BreachMentioned'])), int(bool(e['CompanyMentioned']))]})

trainset = trainset.remove_columns(['BreachMentioned', 'CompanyMentioned'])
testset  = testset.remove_columns(['BreachMentioned', 'CompanyMentioned'])

# Remove mystery unnamed column
trainset = trainset.remove_columns([r for r in trainset.column_names if 'Unnamed' in r])
testset  = testset.remove_columns([r for r in testset.column_names if 'Unnamed' in r])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [47]:
# Sample rows
trainset.shuffle(seed=42)[0]

{'Headline': 'Nasdaq 4000 a prelude to run at 2000 record?7 hours, 56 minutesMarkets',
 'Target': [0, 0]}

In [ ]:
args = TrainingArguments(
    batch_size = (16, 32),
    num_epochs = (1, 16),
    output_dir = '/tmp/lleibm/checkpoints',
)

# f1/accuracy sentence level
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_accuracy_metric = evaluate.load("accuracy", "multilabel")
def compute_metrics(y_pred, y_test):
    return {
        "f1": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
        "accuracy": multilabel_accuracy_metric.compute(predictions=y_pred, references=y_test)["accuracy"],
    }

In [48]:
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2", multi_target_strategy="multi-output")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [49]:
# Train!
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=trainset,
    eval_dataset=testset,
    column_mapping={"Headline": "text", "Target": "label"},
)

/tmp/ipykernel_1289234/1048080675.py:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


/home/lleibm/miniforge3/envs/transformers/lib/python3.11/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [50]:
trainer.train()
metrics = trainer.evaluate()
print(metrics)

***** Running training *****
  Num unique pairs = 2000
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.623400
50,0.194400
100,0.054800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'f1': 0.512, 'accuracy': 0.705}


In [51]:
os.makedirs('./classify/setfit/model', exist_ok=True)
model._save_pretrained('./classify/setfit/model')
# saved_model = SetFitModel._from_pretrained('./classify/setfit/model')